In [1]:
## import modules
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import re
import csv
import numpy as np

In [2]:
#Read review_train.json file
rv_train = []
with open('review_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tra = json.loads(line)
                    rv_train.append(rv_tra)
                    #print(rv_train)
                else:
                    break
        except:
            f.close()

In [3]:
#Read business_train.json file
bs_train = []
with open('business_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    bs_tra = json.loads(line)
                    bs_train.append(bs_tra)
                    #print(bs_train)
                else:
                    break
        except:
            f.close()

In [4]:
# merge review_train.json and business_train.json
 # change the format to dataframe
bs_train_df = pd.DataFrame(bs_train)

In [5]:
import csv

In [6]:
attr_bs = pd.read_csv('atrr_merge.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
business_train = pd.concat([bs_train_df,attr_bs],axis=1)
business_train = business_train.drop(["attributes"],axis=1)

In [8]:
category = business_train[["categories"]]
pat_rest = re.compile(r'.*Gym.*')
num_no_rest = []
for row in tqdm(range(len(category))):
    if category.iloc[row]['categories'] is None:
        num_no_rest.append(row)
    else:
        if not pat_rest.match(category.iloc[row]['categories']):
            num_no_rest.append(row)
business_train = business_train.drop(index = num_no_rest)

100%|██████████| 154606/154606 [00:24<00:00, 6226.16it/s]


In [ ]:
business_train.columns

Index(['business_id', 'categories', 'city', 'hours', 'is_open', 'latitude',
       'longitude', 'name', 'postal_code', 'state', 'Unnamed: 0',
       'AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'Ambience', 'BYOB',
       'BYOBCorkage', 'BestNights', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly',
       'Caters', 'CoatCheck', 'Corkage', 'DietaryRestrictions', 'DogsAllowed',
       'DriveThru', 'GoodForDancing', 'GoodForKids', 'GoodForMeal',
       'HairSpecializesIn', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi'],
      dtype='object')

In [9]:
names = ['AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'Ambience', 'BYOB',
       'BYOBCorkage', 'BestNights', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly',
       'Caters', 'CoatCheck', 'Corkage', 'DietaryRestrictions', 'DogsAllowed',
       'DriveThru', 'GoodForDancing', 'GoodForKids', 'GoodForMeal',
       'HairSpecializesIn', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel',
       'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi']

In [10]:
#Print the attributes with missing values less than 40%
N = 0.4*len(business_train)
ind_nan = []
for i in range(len(names)):  
    if sum(pd.isnull(business_train[names[i]])) < N:
       print(names[i]) 
    else:
       ind_nan.append(names[i])  #Just keep the attributes:BikeParking, BusinessParking, ByAppointmentOnly, GoodForKids

BikeParking
BusinessParking
ByAppointmentOnly
GoodForKids


In [11]:
business_train = business_train.drop(ind_nan,axis = 1)
business_train = business_train.drop(['Unnamed: 0'],axis = 1)

In [ ]:
#business_train.to_csv("business_train.csv" ,index = 0)

In [12]:
id = business_train['business_id']
id = id.as_matrix()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [13]:
#Read review_train.json file
rv_train = []
with open('review_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tra = json.loads(line)
                    rv_train.append(rv_tra)
                    #print(rv_train)
                else:
                    break
        except:
            f.close()
rv_train_df = pd.DataFrame(rv_train) # change the format to dataframe

In [14]:
id_rw = rv_train_df['business_id']
id_rw = id_rw.as_matrix()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [17]:
index = []
for num in range(len(id_rw)):
    if id_rw[num] in id:
        index.append(num)

In [20]:
arr = range(len(rv_train_df))
idx = np.delete(arr,index)
review_train = rv_train_df.drop(index = idx)
review_train

In [30]:
#review_train.to_csv("review_train.csv" ,index = 0)